In [3]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split

data_dir = "music_genre_dataset/Data/images_original/"

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # resize images for AlexNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # normalization
])

dataset = torchvision.datasets.ImageFolder(root=data_dir, transform=transform)
labels = [label for _, label in dataset]

train_data, test_data, train_label, test_label = train_test_split(dataset, labels, test_size=0.2, random_state=42, stratify=labels)


# Create data loaders for training and testing
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)


In [10]:
model = torchvision.models.alexnet(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
model.classifier[6] = torch.nn.Linear(4096, 10)  # 10 classes


In [11]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
device

device(type='cuda')

In [16]:
num_epochs = 10 
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(data_loader)}')

print('Training complete')

Epoch 1/10, Loss: 1.8501302786171436
Epoch 2/10, Loss: 1.522880993783474
Epoch 3/10, Loss: 1.4377997256815434
Epoch 4/10, Loss: 1.3866420239210129
Epoch 5/10, Loss: 1.3119575940072536
Epoch 6/10, Loss: 1.3354383558034897
Epoch 7/10, Loss: 1.2596392650157213
Epoch 8/10, Loss: 1.2419206257909536
Epoch 9/10, Loss: 1.234517516568303
Epoch 10/10, Loss: 1.2786836959421635
Training complete


In [17]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test set: {100 * correct / total}%')


Accuracy on the test set: 65.0%
